# 3 OpenJij + PyQUBO

本稿では，PyQUBOによるコスト関数のQUBOへの変換方法とSimulated Annealing，OpenJijへの変数受け渡しなど についてクリーク被覆問題を例に説明する

まず，PyQUBOのみによるSAの方法を説明する．

**環境構築**
使用する環境は以下の通り．
```
numpy==1.15.4
dimod==0.7.9
six==1.11.0

dwave-cloud-client==0.4.16
dwave-neal==0.4.2
dwave-networkx==0.6.9
dwave-ocean-sdk==1.0.3
dwave-qbsolv==0.2.7
dwave-system==0.5.4
dwavebinarycsp==0.0.10
```
順番を間違えるとバージョンが異なるものに書き換えられる可能性もあるので，最後に
`pip freeze` 
で必ず確認する。

**配列を作成する際の例**

In [ ]:
 x = Array.create('x', shape=4, vartype='BINARY')
```
shape=4で4つの1次元配列をつくる。

ちなみに，２次元配列を作るときは以下のとおり．
```python
>>> from pyqubo import Array, Binary
>>> Array.create('x', shape=(2, 2), vartype='BINARY')
Array([[Binary(x[0][0]), Binary(x[0][1])],
       [Binary(x[1][0]), Binary(x[1][1])]])

 shape=(2, 2)で2×2の上記のような2次元配列を作っている。

***PyQUBOによるSimulated Annealing***
ここから，実際にPyQUBOを用いて最適化問題をSAで解く方法を説明していく．
まずコスト関数をQUBOに変換する方法を紹介する．ここではクリーク被覆問題を例に説明する．

まず，PyQUBOをimport．

In [ ]:
#PyQUBO
from pyqubo import Array, Constraint, Placeholder, solve_qubo

今回はクリーク被覆問題の初期条件を以下のように与える．

In [ ]:
#頂点の数
N_VER = 8
#色の数
N_COLOR = 3
#グラフを定義．0~7とうい名前の頂点があったときに，どの頂点同士が線で結ばれているかを定義している．
graph = [(0,1), (0,2), (1,3), (5,6), (2,3), (2,5), (3,4), (5,7)]

`PyQUBO(N_VER,N_COLOR,graph)` という，PyQUBOを用いてSAした結果を返す関数を定義する．

ここからは，PyQUBO関数の中身について説明していく．

まず，コスト関数に用いるxを以下のように定義．

In [ ]:
    # BINARY変数 PyQUBOでは，以下のようにArray.createとう関数で配列を作ることができる．
    x = Array.create('x', shape=(N_VER,N_VER), vartype='BINARY')

次に，最小化項とペナルティ項を以下のように打ち込み，コスト関数を定義する．
通常のQUBOを作成する方法とはことなり，式をそのまま直感的に打つこむことができる．


In [ ]:
    #最小化項        
    H_A = sum((1-sum(x[v,i] for i in range(1,N_COLOR)))**2 for v in range(N_VER))
    #ペナルティ項
    H_B = sum((-1+sum(x[v,i] for v in range (N_VER)))/2*(sum(x[v,i] for v in range (N_VER))) - sum(x[u,i]*x[v,i] for (u,v) in graph) for i in range (1,N_COLOR))

    #コスト関数
    Q = H_A+H_B

作成したコスト関数は，以下のように`Q.compile().to_qubo()` で簡単にQUBOに変換することができる．

In [ ]:
    # モデルをコンパイル
    model = Q.compile()
    qubo, offset = model.to_qubo()

PyQUBOによってSimulated Annealingするには，
`solve_qubo(qubo)`を用いる．

In [ ]:
    #PyQUBOによるSA
    raw_solution = solve_qubo(qubo)
    # 得られた結果をデコードする
    decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY")

これらの結果をつなげると，以下のようなコードになる．
**クリーク被覆問題 実装例**

In [1]:
# coding: utf-8
#PyQUBO
from pyqubo import Array, Constraint, Placeholder, solve_qubo

#頂点の数
N_VER = 8
#色の数
N_COLOR = 3
#グラフを定義．0~7とうい名前の頂点があったときに，どの頂点同士が線で結ばれているかを定義している．
graph = [(0,1), (0,2), (1,3), (5,6), (2,3), (2,5), (3,4), (5,7)]

def PyQUBO(N_VER,N_COLOR,graph):
    # BINARY変数 PyQUBOでは，以下のようにArray.createとう関数で配列を作ることができる．
    x = Array.create('x', shape=(N_VER,N_VER), vartype='BINARY')

    #最小化項        
    H_A = sum((1-sum(x[v,i] for i in range(1,N_COLOR)))**2 for v in range(N_VER))
    #ペナルティ項
    H_B = sum((-1+sum(x[v,i] for v in range (N_VER)))/2*(sum(x[v,i] for v in range (N_VER))) - sum(x[u,i]*x[v,i] for (u,v) in graph) for i in range (1,N_COLOR))
    #コスト関数
    Q = H_A+H_B 
    
    # モデルをコンパイル
    model = Q.compile()
    qubo, offset = model.to_qubo()
    
    #PyQUBOによるSA
    raw_solution = solve_qubo(qubo)
    # 得られた結果をデコードする
    decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY")
    return decoded_solution


resultPyQUBO = PyQUBO(N_VER,N_COLOR,graph)
print(resultPyQUBO)

{'x': {0: {1: 0, 2: 0}, 1: {1: 0, 2: 0}, 2: {1: 0, 2: 1}, 3: {1: 0, 2: 1}, 4: {1: 0, 2: 0}, 5: {1: 1, 2: 0}, 6: {1: 1, 2: 0}, 7: {1: 0, 2: 0}}}


***PyQUBOによってQUBOに変換したモデルのOpenJijによる評価***
次に，上のコードに追記する形で，OpenJijにQUBOを受け渡し，SQAで解く方法を以下で紹介する．
まずPyQUBO関数の中で，QUBOも返すように変更する．
   ` return qubo,decoded_solution`

そして，関数の外で，以下のようにQUBOを受け渡すことでOpenJijによる評価ができるようになる．

In [ ]:
#quboをQという変数に格納．
Q,resultPyQUBO = PyQUBO(N_VER,N_COLOR,graph)

#あとは通常のQUBOを作成してOpenJijで評価する場合と同じ．
sampler = oj.SQASampler(iteration=10, step_num=100)
response = sampler.sample_qubo(Q)
#結果を出力．
print(response.states)

まとめると，以下のようになる．

In [2]:
# coding: utf-8
#openJijを使用。
import openjij as oj
#PyQUBO
from pyqubo import Array, Constraint, Placeholder, solve_qubo

#頂点の数
N_VER = 8
#色の数
N_COLOR = 3
graph = [(0,1), (0,2), (1,3), (5,6), (2,3), (2,5), (3,4), (5,7)]
def PyQUBO(N_VER,N_COLOR,graph):
    #イジング変数の数
    N = N_VER * N_COLOR
    # BINARY変数
    x = Array.create('x', shape=(N_VER,N_VER), vartype='BINARY')
    #最小化項        
    H_A = sum((1-sum(x[v,i] for i in range(1,N_COLOR)))**2 for v in range(N_VER))
    #ペナルティ項
    H_B = sum((-1+sum(x[v,i] for v in range (N_VER)))/2*(sum(x[v,i] for v in range (N_VER))) - sum(x[u,i]*x[v,i] for (u,v) in graph) for i in range (1,N_COLOR))
    #コスト関数
    Q = H_A+H_B 
    
    # モデルをコンパイル
    model = Q.compile()
    qubo, offset = model.to_qubo()
    
    #PyQUBOによるSA
    raw_solution = solve_qubo(qubo)
    # 得られた結果をデコードする
    decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY")
    #ここでQUBOも返すように変更する．
    return qubo,decoded_solution
    

Q,resultPyQUBO = PyQUBO(N_VER,N_COLOR,graph)
sampler = oj.SASampler(iteration=10, step_num=100)
response = sampler.sample_qubo(Q)

#PyQUBOによるSAの結果出力
#print(resultPyQUBO)

#OpenJijによるSQAの結果出力
print(response.states)

[[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0]]


参考：PyQUBO公式ドキュメント
https://pyqubo.readthedocs.io/en/latest/reference/array.html?highlight=arry%20create

※余談として，.pyipynbは.pyファイルに以下のように変換できるよう．
`$ jupyter nbconvert --to script [YOUR_NOTEBOOK].ipynb`